In [1]:
import os
import time
import requests
import datetime
import numpy as np
import pandas as pd
from io import StringIO
from statsmodels.tsa.base.datetools import dates_from_str

#visual
import matplotlib.pyplot as plt

#import mplfinance as mpf
%matplotlib inline
import talib
from statsmodels.tsa.base.datetools import dates_from_str

In [2]:
class data:
    
    def stock_num(self):  #股票清單

        #上市
        url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
        response = requests.get(url)
        listed = pd.read_html(response.text)[0]
        listed.columns = listed.iloc[0,:]
        listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
        listed = listed.iloc[1:]

        #上櫃
        url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=4&industry_code=&Page=1&chklike=Y"
        response = requests.get(url)
        OTC = pd.read_html(response.text)[0]
        OTC.columns = OTC.iloc[0,:]
        OTC = OTC[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
        OTC = OTC.iloc[1:]

        stock_1 = listed["有價證券代號"]
        stock_2 = OTC["有價證券代號"]
        stock_num1 = stock_1.apply(lambda x: str(x) + ".TW")
        stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
        stock_num = pd.concat([stock_num1,stock_num2])
        stock_num.reset_index(drop = True,inplace = True)

        return stock_num


    def df_m(self,df):  #轉成月資料 取每月第一天

        df = df.reset_index()

        df_month = pd.DataFrame(columns = ['Date','Open','High','Low','Close','Volume'])

        year1 = int(str(df['Date'][df.index[0]]).split('-')[0])
        year2 = int(str(df['Date'][df.index[-1]]).split('-')[0])
        month1 = int(str(df['Date'][df.index[0]]).split('-')[1])
        month2 = int(str(df['Date'][df.index[-1]]).split('-')[1])

        for y in range(year1,year2+1):
            if year1 < year2:
                for m in range(12):
                    if month1 == 12:

                        time1 = '-'.join([str(year1),str(month1)])
                        time2 = '-'.join([str(year1+1),str(1)])
                        df_month = self.catch_mdata(df,df_month,time1,time2,y,m)

                        month1 = 1
                        year1 += 1

                        break
                    else:

                        time1 = '-'.join([str(year1),str(month1)])
                        time2 = '-'.join([str(year1),str(month1+1)])
                        df_month = self.catch_mdata(df,df_month,time1,time2,y,m)

                        month1 += 1

            elif year1 == year2:                      
                for m in range(month2):
                    if month1 == 12:

                        time1 = '-'.join([str(year1),str(month1)])
                        time2 = '-'.join([str(year1+1),str(1)])
                        df_month = self.catch_mdata(df,df_month,time1,time2,y,m)

                        break
                    else:

                        time1 = '-'.join([str(year1),str(month1)])
                        time2 = '-'.join([str(year1),str(month1+1)])
                        df_month = self.catch_mdata(df,df_month,time1,time2,y,m)

                        month1 += 1

            else:
                break

        df_month['Date'] = pd.to_datetime(df_month['Date'])
        df_month.reset_index(inplace=True,drop=True)

        return df_month

    def catch_mdata(self,df,df_month,time1,time2,y,m):
        df_m_now = df.loc[(df['Date'] >= pd.to_datetime(time1)) & 
                          (df['Date'] < pd.to_datetime(time2))]

        df_month.loc[f'{y}.{m+1}','Date'] = time1
        df_month.loc[f'{y}.{m+1}','Open'] = df_m_now.iloc[0,1]
        df_month.loc[f'{y}.{m+1}','High'] = max(df_m_now['High'])          
        df_month.loc[f'{y}.{m+1}','Low'] = min(df_m_now['Low'])     
        df_month.loc[f'{y}.{m+1}','Close'] = df_m_now.iloc[-1,4]
        df_month.loc[f'{y}.{m+1}','Volume'] = sum(df_m_now['Volume'])

        return df_month

    ##讀寫成csv檔，新檔案直接覆蓋
    def stock_data(self,stock_id,time_start,time_end) :
        days = 24 * 60 * 60    #一天有86400秒 
        initial = datetime.datetime.strptime( '1970-01-01' , '%Y-%m-%d' )
        start = datetime.datetime.strptime( time_start , '%Y-%m-%d' )
        end = datetime.datetime.strptime( time_end, '%Y-%m-%d' )
        period1 = start - initial
        period2 = end - initial
        s1 = period1.days * days
        s2 = period2.days * days
        url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=" + str(s1) + "&period2=" + str(s2) + "&interval=1d&events=history&includeAdjustedClose=true"
        headers = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
    }
        response = requests.get(url,headers = headers)
        df = pd.read_csv(StringIO(response.text),index_col = "Date",parse_dates = ["Date"])
                         
        address = r"./stockdata/" + stock_id[:4] + ".csv"
        df.to_csv(address,encoding='utf-8')

        df_month = self.df_m(df)
        address_m = r"./stockdata_m/" + stock_id[:4] + ".csv"
        df_month.to_csv(address_m,encoding='utf-8',index=False)
        
    def income_m(self,year,month) -> int: #抓月營收
        if year > 1990:
            year -= 1911
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
        if year <= 98:
            url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'

        # 偽瀏覽器
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

        # 下載該年月的網站，並用pandas轉換成 dataframe
        r = requests.get(url, headers=headers)
        r.encoding = 'big5'

        dfs = pd.read_html(StringIO(r.text), encoding='big-5')

        df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])

        if 'levels' in dir(df.columns):
            df.columns = df.columns.get_level_values(1)
        else:
            df = df[list(range(0,10))]
            column_index = df.index[(df[0] == '公司代號')][0]
            df.columns = df.iloc[column_index]

        try:
            df.drop('備註',axis=1,inplace=True)
        except:
            pass
        
        df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
        df = df[~df['當月營收'].isnull()]
        df = df[(df['公司代號'] != '合計') & (df['公司代號'] != '全部國內上市公司合計')]
        df.set_index('公司代號',inplace=True)

        return df
    
    def create_date(self,year,month) -> str:
        year = int(year)
        month = int(month)
        if month < 10:
            return str(year)+'-0'+str(month)
        else:
            return str(year)+'-'+str(month)
    
    def check_date(self):
        df = pd.read_csv(r'.\stockdata\1101.csv')
        year1,mon1,year_last,mon_last = df['Date'][0][:4],df['Date'][0][5:7],df['Date'][len(df['Date'])-1][:4],df['Date'][len(df['Date'])-1][5:7]
        year1 = int(year1)
        mon1 = int(mon1)
        year_last = int(year_last)
        mon_last = int(mon_last)
        return year1,mon1,year_last,mon_last